In [2]:
import re
import numpy as np
import pickle
from pathlib import Path
from collections import Counter

In [3]:
def parse_title(title):
    if len(title) < 3:
        return True
    if title[:2] == "Me":
        return True
    else:
        return False

def clean_message(msg):
    msg = msg.replace('\n', ' ').lower()
    msg = msg.replace("\xc2\xa0", "")
    msg = re.sub('([\(\).,!?])', "", msg)
    msg = re.sub(" +"," ", msg)
    return msg

In [4]:
data_in = Path("data") / "sms.txt"
lines = []
with open(data_in, 'r', encoding='utf-8') as f:
    while True:
        title = f.readline()
        if not title:
            break
        content = f.readline()
        if not content:
            break
        ok = True
        while (ok):
            s = f.readline() # whitespace
            if not s:
                break
            if len(s)==1:
                ok = False
            else:
                content = content + s
        lines.append((parse_title(title), clean_message(content)))

In [5]:
lines = [(t,l) for t, l in lines if l != "[attachment] "]

In [5]:
conversation_dict = {}
other_message = ""
my_message = ""
for (is_me, line) in lines:
    if not is_me:
        # commit the previous message if it is ready
        if my_message != "" and other_message != "":
            conversation_dict[other_message] = my_message
            my_message = ""
            other_message = ""
        other_message = other_message + line
        continue
    # if it is my message
    my_message = my_message + line

In [6]:
with open(Path("data") / "conversation.txt", 'w', encoding='utf-8') as f:
    for k,v in conversation_dict.items():
        f.write(k + "\n")
        f.write(v + "\n")
np.save(Path('data') / "conversation_dict.npy", conversation_dict)

In [7]:
with open(Path("data") / "conversation.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    c = Counter()
    for line in lines:
        c.update(line[:-1].split(" "))
word_list = list(c.keys())
word_list.append("<padding>")
word_list.append("<eos>")
with open(Path("data") / "word_list.txt", "wb") as f:
    pickle.dump(word_list, f)

## Experiment

The idea is to make another bot that imitates the other person's conversation style and pitch the two bots together.

In [14]:
def parse_title(title):
    if len(title) < 3:
        return True
    if title[:2] == "Me":
        return False
    else:
        return True

In [15]:
data_in = Path("data") / "sms.txt"
lines = []
with open(data_in, 'r', encoding='utf-8') as f:
    while True:
        title = f.readline()
        if not title:
            break
        content = f.readline()
        if not content:
            break
        ok = True
        while (ok):
            s = f.readline() # whitespace
            if not s:
                break
            if len(s)==1:
                ok = False
            else:
                content = content + s
        lines.append((parse_title(title), clean_message(content)))

In [16]:
lines = [(t,l) for t, l in lines if l != "[attachment] "]

In [21]:
conversation_dict = {}
other_message = ""
my_message = ""
for (is_me, line) in lines:
    if not is_me:
        # commit the previous message if it is ready
        if my_message != "" and other_message != "":
            conversation_dict[other_message] = my_message
            my_message = ""
            other_message = ""
        other_message = other_message + line
        continue
    # if it is my message
    my_message = my_message + line

In [22]:
with open(Path("data") / "conversation_reverse.txt", 'w', encoding='utf-8') as f:
    for k,v in conversation_dict.items():
        f.write(k + "\n")
        f.write(v + "\n")
np.save(Path('data') / "conversation_dict_reverse.npy", conversation_dict)

In [23]:
with open(Path("data") / "conversation_reverse.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    c = Counter()
    for line in lines:
        c.update(line[:-1].split(" "))
word_list = list(c.keys())
word_list.append("<padding>")
word_list.append("<eos>")
with open(Path("data") / "word_list_reverse.txt", "wb") as f:
    pickle.dump(word_list, f)